In [1]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '成品仓库进出明细'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import sys
month=' ('+str(sys.argv[1])+'月)'
#month =' (6月)'   

file_name = '成品仓库进出明细'
input_filename = file_name +'.xls'
output_filename = folder_name + '/'+ file_name+ '_添加类别.xlsx'

df = pd.read_excel(input_filename,sheet_name = 0)
df.columns.values


array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计', 'Unnamed: 18',
       '出库统计', 'Unnamed: 20'], dtype=object)

In [2]:
#清除 "入库统计"
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_entering_warehouse = col_names.index('入库统计')
str_entering_warehouse_1= col_names[idx_entering_warehouse]
str_entering_warehouse_2 = col_names[idx_entering_warehouse +1]

print(str_entering_warehouse_1)
print(str_entering_warehouse_2)

df.drop([str_entering_warehouse_1,str_entering_warehouse_2], axis=1, inplace = True)


入库统计
Unnamed: 18


In [3]:
#重命名"类别名称"所在列 的标签
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_typename = tag_names.index('类别名称')
str_typename = col_names[idx_typename]
print(str_typename,'\n')

df.rename(index=str, columns={str_typename: 'type_name'},inplace=True)


str_typesize = col_names[idx_typename + 1]
print(str_typesize, '\n')

df.rename(index=str, columns={str_typesize: 'pd_size'}, inplace = True)

# 添加一列，命名为 type
print(df.columns.values,'\n')
df.insert(idx_typename,'type',df.type_name)
df.columns.values

# drop last line for summary information
df.drop(df.index[len(df)-1], inplace=True)


Unnamed: 8 

Unnamed: 9 

['单据' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4' '属性'
 'Unnamed: 6' 'Unnamed: 7' 'type_name' 'pd_size' 'Unnamed: 10'
 'Unnamed: 11' 'Unnamed: 12' 'Unnamed: 13' 'Unnamed: 14' 'Unnamed: 15'
 'Unnamed: 16' '出库统计' 'Unnamed: 20'] 



In [4]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'XF'] = '地砖'

df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name == 'PG'] = '瓷片'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'

df.type[df.type_name =='HM'] = '木纹砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == 'HNW'] = '镜面砖'
df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '木托'] = '木托'
df.type[df.type_name == '捆砖带'] = '捆砖带'
df.type[df.type_name == '换包装'] = '换包装'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'
df.type[df.type_name == '瓷砖胶'] = '5KG'
df.type[df.type_name == '圣陶居'] = '圣陶居'
df.type[df.type_name == '展美'] = '展美'


# update 2019/07/04
new_type_list = ['BO1A','BO1B','BO2B',
                'FG1A','FG1B','FG2A','FG2B','FO1A','FO1B','F02A','FO2B',
                'PG1A','PG1B','PG1C','PG2A','PG2B','PX1A','TO1A','T01B',
                'FU2A','GO1A',
                'PX1A','FU02A','PG1B','PG2B']

for type_name in new_type_list:
    df.type[df.type_name == type_name] = '瓷片'


# update 2019/09/02
new_type_list = ['PO1A', 'PO1C','TO2B']
for type_name in new_type_list:
    df.type[df.type_name == type_name] = '瓷片'


print(df.groupby(['type','pd_size']).type.agg('count'))

type   pd_size
FU2A   300X600       2
GO1A   300X600       2
五金仓    1            18
其它费用   1             4
圣陶居    800X800      24
地砖     300X300     106
       300X600       4
       400X800       1
捆砖带    1             7
换包     1             2
换托费    1             1
搬运     1            12
木托     110X110      10
木纹砖    300X600       1
欧驰     800X800       1
浮雕     60X600        1
       70X600       12
       75X600        5
       80X600       11
瓷片     300X300     272
       300X600    1256
       400X800     385
类别     规格            1
纳米瓷砖胶  5KG           2
腰线     60X600        3
       70X300        6
       70X600       13
       80X300        1
花片     300X600      53
运费     1             8
金刚石    800X800       2
镜面砖    400X800       3
Name: type, dtype: int64


In [13]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('编号',inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [14]:
## regroup data
df2 = output_df.copy(deep=True)

# fill nan with ‘空白’
df2['销售类型'] = df2['销售类型'].fillna('未注明')

# fill nan for some classification term 
df2['仓库名称'] = df2['仓库名称'].fillna('未注明')
df2['品牌'] = df2['品牌'].fillna('未注明')

print(df2['销售类型'].unique())
print( df2.columns.values )

['未注明' '正常销售' '折价销售' '免费样板' '销售费用' '免费费用' '免费广告' '收费样板' '物资自用']
['日期' '类型' '销售类型' '往来单位' '仓库名称' '产品编号' '产品名称' '类别' '类别名称' '规格' '庄数' '色号'
 '等级' '品牌' '包装' '单位' '单价' '数量(片)' '金额']


In [15]:
# 销售成本 : '正常销售 + 折价销售 + 收费样板 + 销售调价

#sales = df2[ (df2['销售类型'] == '正常销售') | (df2['销售类型'] == '折价销售') | (df2['销售类型'] == '收费样板') | (df2['销售类型'] == '销售调价') ]
sales = df2[ (df2['销售类型'] != '挂账样板') & 
             (df2['销售类型'] != '挂账广告') & 
             (df2['销售类型'] != '免费广告') & 
             (df2['销售类型'] != '免费样板') ]
             #(df2['销售类型'] != '免费费用') ]


# 挂账样板

model = df2[ df2['销售类型'] == '挂账样板' ]

# 挂账广告

adv = df2[ df2['销售类型'] == '挂账广告']

# 免费广告

free_adv = df2[ df2['销售类型'] == '免费广告']

# 免费样板

free_model = df2[ df2['销售类型'] == '免费样板']

# 免费费用

#free_expenses = df2[ df2['销售类型'] == '免费费用']


In [16]:
def handle_worksheet_format(writer, group, title):
    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']      
    # set data format
    for idx, col in enumerate(group):  # loop through all columns
        series = group[col]
        if isinstance(series[0],str): char_ratio = 2 
        else: char_ratio = 1 
        max_len = max((
                        series.astype(str).map(len).max() * char_ratio ,  # len of largest item
                        len( str(series.name) * 2)  # len of column name/header
                    )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = group.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name)* 2)  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)

    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})      
    worksheet_cols = group.shape[1]
    range_str = 'A1:'+chr(ord('A')+worksheet_cols)+'1'
    worksheet.merge_range(range_str,title,header_format)    

In [17]:
def summary_product_category(key, df_temp, out_path=None, title=None):
    if out_path is None:
        output_filename = folder_name +'/summary_'+key+'.xlsx'
        output_filename2 = folder_name +'/往来单位_'+key+'.xlsx'            
    else:
        output_filename = out_path + '/summary_'+key+'.xlsx'
        output_filename2 = out_path +'/往来单位_'+key+'.xlsx'
    
    if title is None:
        title = key+month       

    group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                        df_temp.groupby('类别')['金额'].agg(np.sum)],
                       axis = 1).sort_values(by='金额',ascending = False)

    group.loc[group.shape[0]] = group.sum()
    group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)
    group = group.round(2)

    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
    group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

    handle_worksheet_format(writer, group, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
    ########################
    # 往来单位
    ########################
    group2 = (df_temp.groupby([df_temp['往来单位'].fillna('空白')])
                    .agg({'金额':'sum'})
                    .reset_index()
                    .set_index('往来单位')
                    .sort_values(by='金额',ascending = False)
             )

    group2.loc[group2.shape[0]] = group2.sum()
    group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)
    group2 = group2.round(2)

    writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
    group2.to_excel(writer,startrow = 1, sheet_name = 'sheet1') #creates workbook 

    # handle the spreadsheet format
    
    title = key + ' 往来单位' + month
    handle_worksheet_format(writer, group2, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
    return



In [18]:
key = '销售成本'
summary_product_category(key, sales)

#key ='挂账样板'
#summary_product_category(key, model)

key = '免费样板'
summary_product_category(key, free_model)

key = '免费广告'
summary_product_category(key, free_adv)

#key = '免费费用'
#summary_product_category(key, free_expenses)


In [19]:
# 不同销售类型的金额汇总
def summary_by_sales_type(df2, out_path=None, title=None,term=None):
    if out_path is None:
        output_filename = folder_name + '/不同销售类型金额汇总.xlsx'
    else:
        if term is None:
            output_filename = out_path + '/不同销售类型金额汇总.xlsx'
        else:
            output_filename = out_path + '/不同销售类型金额汇总_'+term+'.xlsx'

    if title is None:
        title = '销售类型金额统计'+month
   
    group = (df2.groupby('销售类型')['金额']
               .agg(np.sum)
               .to_frame()
               .sort_values(by='金额',ascending = False)
            )
    
    # add some additional sub-category
    drop_list = ['免费样板', '免费广告', '挂账样板', '挂账广告']
    sales_cost_index=  group.index
    for x in drop_list:
        if x in sales_cost_index:
            group.loc['统计：'+x] = group.loc[x]
            sales_cost_index = sales_cost_index.drop(x)

    sub_category={'统计：销售成本' : sales_cost_index}

    if sub_category:
        for key, values in sub_category.items():
            group.loc[key] = [0] * group.shape[1]
            for product in values:
                if product in group.index:
                    group.loc[key] = [sum(x) for x in zip(group.loc[key], group.loc[product])]

    #save to 2 digits precision float
    group = group.round(2)
    
    #write to excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
    group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

    # handle the spreadsheet format
    handle_worksheet_format(writer, group, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    

In [20]:
# 不同销售类型的金额汇总
summary_by_sales_type(df2)

# 按 仓库/品牌 归类

In [21]:
# testing code

df_temp=sales

term_list = df_temp['仓库名称'].unique()
print(term_list)
df_term_list = [df_temp[df_temp['仓库名称']==term] for term in term_list]
df_term_list = [df_temp] + df_term_list

data_list=  [pd.concat([ x.groupby('类别')['数量(片)'].agg(np.sum),
                        x.groupby('类别')['金额'].agg(np.sum)],
                       axis = 1
                      ).sort_values(by='金额',ascending = False)
            for x in df_term_list]

data_list[0].columns = [['所有仓库','所有仓库'], data_list[0].columns]
for i in range(len(term_list)):
    data_list[i+1].columns = [ [term_list[i],term_list[i]], data_list[i+1].columns]        
    
grouped = pd.concat(data_list,axis=1,sort=False).fillna(0.00)
summation = grouped.sum()
summation.name = '总计'
grouped = grouped.append(summation)
grouped

['恒希仓' '佛山仓' '南庄仓' '广西仓' '外仓' '样板仓' '五金(新)' '佛山2仓']


所有仓库                    恒希仓                佛山仓               南庄仓  \
          数量(片)          金额      数量(片)         金额   数量(片)        金额   数量(片)   
瓷片    299305.00  1228836.36  144007.00  554274.25     0.0      0.00  4376.0   
圣陶居     2365.00    47798.70       0.00       0.00   555.0  11442.20     0.0   
木托       130.00     8450.00       3.00     195.00     0.0      0.00     0.0   
地砖      2235.00     4723.14     615.00    1270.89     0.0      0.00   375.0   
搬运     17556.00     4433.46   15665.00    4016.19  1433.0    302.86   458.0   
东晟       832.00     3152.00       0.00       0.00     0.0      0.00     0.0   
花片       160.00     2444.36       0.00       0.00   160.0   2444.36     0.0   
腰线       455.00     2216.89       0.00       0.00   453.0   2216.89     0.0   
浮雕       172.00     1678.00       0.00       0.00   172.0   1678.00     0.0   
木纹砖      495.00     1138.50       0.00       0.00   495.0   1138.50     0.0   
宝柏龙       64.00      480.00       0.00       0.00     0.0      0.00     0.0   
外购       133.00      388.00       0.00       0.00   133.0    388.00     0.0   
其它费用      90.00      311.00       0.00       0.00    18.0     95.00     0.0   
标签费     2000.00      200.00       0.00       0.00     0.0      0.00     0.0   
纸箱        55.00      131.50       0.00       0.00    45.0    112.50     0.0   
五金仓      100.00      100.89       0.00       0.00     0.0      0.00     0.0   
换包        34.00       95.20       0.00       0.00    34.0     95.20     0.0   
换托费      184.00       16.56       0.00       0.00     0.0      0.00     0.0   
运费       450.36        0.01     450.36       0.01     0.0      0.00     0.0   
捆砖带        0.00        0.00       0.00       0.00     0.0      0.00     0.0   
总计    326815.36  1306594.57  160740.36  559756.34  3498.0  19913.51  5209.0   

                     广西仓                 外仓            样板仓       五金(新)  \
            金额     数量(片)         金额   数量(片)       金额 数量(片)   金额  数量(片)   
瓷片    19291.70  150911.0  655270.41     0.0      0.0  11.0  0.0    0.0   
圣陶居       0.00       0.0       0.00  1810.0  36356.5   0.0  0.0    0.0   
木托        0.00     127.0    8255.00     0.0      0.0   0.0  0.0    0.0   
地砖      900.00    1245.0    2552.25     0.0      0.0   0.0  0.0    0.0   
搬运      114.41       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
东晟        0.00       0.0       0.00   832.0   3152.0   0.0  0.0    0.0   
花片        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
腰线        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
浮雕        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
木纹砖       0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
宝柏龙       0.00       0.0       0.00    64.0    480.0   0.0  0.0    0.0   
外购        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
其它费用      0.00      72.0     216.00     0.0      0.0   0.0  0.0    0.0   
标签费       0.00    2000.0     200.00     0.0      0.0   0.0  0.0    0.0   
纸箱        0.00      10.0      19.00     0.0      0.0   0.0  0.0    0.0   
五金仓       0.00       0.0       0.00     0.0      0.0   0.0  0.0  100.0   
换包        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
换托费       0.00     184.0      16.56     0.0      0.0   0.0  0.0    0.0   
运费        0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
捆砖带       0.00       0.0       0.00     0.0      0.0   0.0  0.0    0.0   
总计    20306.11  154549.0  666529.22  2706.0  39988.5  11.0  0.0  100.0   

              佛山2仓       
          金额 数量(片)   金额  
瓷片      0.00   0.0  0.0  
圣陶居     0.00   0.0  0.0  
木托      0.00   0.0  0.0  
地砖      0.00   0.0  0.0  
搬运      0.00   0.0  0.0  
东晟      0.00   0.0  0.0  
花片      0.00   0.0  0.0  
腰线      0.00   2.0  0.0  
浮雕      0.00   0.0  0.0  
木纹砖     0.00   0.0  0.0  
宝柏龙     0.00   0.0  0.0  
外购      0.00   0.0  0.0  
其它费用    0.00   0.0  0.0  
标签费     0.00   0.0  0.0  
纸箱      0.00   0.0  0.0  
五金仓   100.89   0.0  0.0  
换包      0.00   0.0  0.

In [22]:
def product_classify_by_terms(term, term_string, key, df_temp):
    term_list = df_temp[term_string].unique()
    print(key,':',term_list)
    
    for name in term_list:
        df_item = df_temp[df_temp[term_string]==name]        
        out_path =  term+ '/'+ name + '/' + folder_name
        if not os.path.exists(out_path):
            os.makedirs(out_path)    

        name_str = key + '_' + name
        title= key+ ':' + name + month
        summary_product_category(name_str, df_item, out_path, title)

        
def summary_sales_types_classify_by_terms(term, term_string, df_temp):
    term_list = df_temp[term_string].unique()
    
    for name in term_list:
        df_item = df_temp[df_temp[term_string]==name]
        out_path =  term+ '/金额统计_'+folder_name
        if not os.path.exists(out_path):
            os.makedirs(out_path) 
            
        title = '金额统计:'+name+ month
        summary_by_sales_type(df2, out_path, title, name)     

In [23]:
def handle_worksheet_format(writer, df_temp, title):
    workbook = writer.book
    worksheet = writer.sheets['sheet1']    
    
    for idx, col in enumerate(df_temp):  # loop through all columns
        series = df_temp[col]
        max_len = max(( series.astype(str).map(len).max() ,  # len of largest item
                        max( [len(str(x)) for x in series.name])*2  # len of column name/header
                      )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = df_temp.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name))  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)
    
    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})
    worksheet_cols = df_temp.shape[1]+1
    if (worksheet_cols <=25):
        range_str = 'A1:'+chr(ord('A')+worksheet_cols-1)+'1'
    else:
        last_col_str = chr(ord('A')+ worksheet_cols // 26 - 1) + chr(ord('A')+ (worksheet_cols-1) % 26) + '1'
        range_str = 'A1:'+last_col_str    
    
    worksheet.merge_range(range_str,title,header_format) 
    

def summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title=None):
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    output_filename = out_path + '/summary_'+key+'_'+term+'分类.xlsx'      
    if title is None:
        title=key+month
    
    term_list = df_temp[term_string].unique()
    if ('未注明' in term_list):
        term_list= term_list.tolist()
        term_list.remove('未注明')
        term_list.append('未注明')
        
    df_term_list = [df_temp[df_temp[term_string]==name] for name in term_list]
    df_term_list = [df_temp] + df_term_list

    data_list=  [pd.concat([ x.groupby('类别')['数量(片)'].agg(np.sum),
                            x.groupby('类别')['金额'].agg(np.sum)],
                           axis = 1
                          ).sort_values(by='金额',ascending = False)
                for x in df_term_list]
    
    data_list[0].columns = [['所有'+term,'所有'+term], data_list[0].columns]
    for i in range(len(term_list)):
        data_list[i+1].columns = [ [term_list[i],term_list[i]], data_list[i+1].columns]        

    grouped = pd.concat(data_list,axis=1,sort=False).fillna(0.00)
    summation = grouped.sum()
    summation.name = '总计'
    grouped = grouped.append(summation)
    
    # round to precison of 2 digits
    grouped = grouped.round(2)
    
    # write to Excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workbook
    grouped.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook    
    
    
    # handle the spreadsheet format
    handle_worksheet_format(writer, grouped, title)       

    writer.save() #saves workbook to file in python file directory
    writer.close()
    

In [24]:
term ='仓库'
term_string = '仓库名称'
term_list = df2[term_string].unique()
print('所有'+term+'名称:',term_list)

out_path =  term+ '/' + folder_name 
def summary_product_by_factory_and_category(key, df_temp, out_path, title=None):
    summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title)

def product_by_factory(key, df_temp):
    product_classify_by_terms(term, term_string, key, df_temp)
    
    
key = '销售成本'
product_by_factory(key, sales)
summary_product_by_factory_and_category(key, sales, out_path, key)

#key ='挂账样板'
#product_by_factory(key, model)
#summary_product_by_factory_and_category(key, model, out_path, title)

key = '免费样板'
product_by_factory(key, free_model)
summary_product_by_factory_and_category(key, free_model, out_path, key)

key = '免费广告'
product_by_factory(key, free_adv)
summary_product_by_factory_and_category(key, free_adv, out_path, key)

#key = '免费费用'
#product_by_factory(key, free_expenses)
#summary_product_by_factory_and_category(key, free_expenses, out_path, key)


def summary_by_sales_type_and_factory(df_temp):
    summary_sales_types_classify_by_terms(term, term_string, df_temp)

summary_by_sales_type_and_factory(df_temp)

所有仓库名称: ['恒希仓' '佛山仓' '南庄仓' '广西仓' '样板仓' '外仓' '五金(新)' '佛山2仓']
销售成本 : ['恒希仓' '佛山仓' '南庄仓' '广西仓' '外仓' '样板仓' '五金(新)' '佛山2仓']
免费样板 : ['广西仓' '样板仓' '恒希仓' '南庄仓' '佛山仓']
免费广告 : ['五金(新)']


In [25]:
term ='品牌'
term_string = '品牌'
term_list = df2[term_string].unique()
print('所有'+term+'名称:',term_list)

out_path =  term+ '/' + folder_name 
def summary_product_by_brand_and_category(key, df_temp, out_path, title=None):
    summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title)

def product_by_brand(key, df_temp):
    product_classify_by_terms(term, term_string, key, df_temp)
    
key = '销售成本'
product_by_brand(key, sales)
summary_product_by_brand_and_category(key, sales, out_path)

#key ='挂账样板'
#product_by_brand(key, model)
#summary_product_by_brand_and_category(key, model, out_path)

key = '免费样板'
product_by_brand(key, free_model)
summary_product_by_brand_and_category(key, free_model, out_path)

key = '免费广告'
product_by_brand(key, free_adv)
summary_product_by_brand_and_category(key, free_adv, out_path)

#key = '免费费用'
#product_by_brand(key, free_expenses)
#summary_product_by_brand_and_category(key, free_expenses, out_path)


def summary_by_sales_type_and_brand(df_temp):
    summary_sales_types_classify_by_terms(term, term_string, df_temp)

summary_by_sales_type_and_brand(df_temp)

所有品牌名称: ['星晴' '维万家' '恒希' '佛山' '恒希样板' '出口' '广西厂' '无']
销售成本 : ['星晴' '维万家' '恒希' '佛山' '出口' '广西厂' '无' '恒希样板']
免费样板 : ['恒希样板' '星晴' '维万家' '恒希']
免费广告 : ['恒希' '维万家']


In [26]:
print('success')

success
